In [ ]:
import os
import sys
import pandas as pd

app_name = '___' 
assert app_name is not None, "Specify app_name"

import pyspark.sql.functions as F
from pyspark import SparkContext, SparkConf, HiveContext
from pyspark.sql import SparkSession, DataFrame
from pyspark.sql.functions import col, asc, desc, length, udf, round, monotonically_increasing_id
from pyspark.sql.types import ArrayType, StringType, FloatType, IntegerType


In [4]:
from IPython.core.display import display, HTML
display(HTML("<style> .container { width:80% !important;}</style>"))
display(HTML("<style> div#header { width:100%; }</style>"))
display(HTML('<style>pre { white-space: pre !important; }</style>'))

pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', 150)

/tmp/ipykernel_465028/2922149680.py:1: DeprecationWarning: Importing display from IPython.core.display is deprecated since IPython 7.14, please import from IPython display
  from IPython.core.display import display, HTML


In [ ]:
os.environ['SPARK_MAJOR_VERSION'] = '3'
os.environ['SPARK_HOME'] = '/usr/sdp/current/spark3-client/'
# os.environ['PYSPARK_DRIVER_PYTHON'] = 'python'
os.environ['PYSPARK_PYTHON'] = '/opt/sdp/stockfish/bin/python'
os.environ['PYSPARK_DRIVER_PYTHON'] = '/opt/sdp/stockfish/bin/python'
os.environ['LD_LIBRARY_PATH'] = '/opt/python/virtualenv/jupyter/lib'
sys.path.insert(0, '/usr/sdp/current/spark3-client/python/')
sys.path.insert(0, '/usr/sdp/current/spark3-client/python/lib/py4j-0.10.9.3-src.zip')

In [6]:
conf = SparkConf().setAppName(app_name)\
    .setMaster("yarn")\
    .set('spark.dynamicAllocation.enabled', 'True')\
    .set('spark.executor.memory','48g')\
    .set('spark.driver.memory','48g')\
    .set('spark.executor.memoryOverhead','36g')\
    .set('spark.driver.maxResultSize','512g')\
    .set('spark.executor.cores', '15')\
    .set('spark.driver.maxResultSize','512g')\
    .set('spark.dynamicAllocation.initialExecutors', 1)\
    .set('spark.dynamicAllocation.minExecutors', 1)\
    .set('spark.dynamicAllocation.maxExecutors', 10)\
    .set('spark.port.maxRetries', '150')\
    .set('spark.driver.cores', '1')\
    .set('spark.driver.memoryOverhead', '128g')\
    .set('spark.shuffle.service.enabled', 'true')\
    .set('spark.sql.parquet.datetimeRebaseModeInRead', 'CORRECTED')\
    .set('spark.sql.parquet.int96RebaseModeInWrite', 'CORRECTED')\
    .set('spark.sql.parquet.int96RebaseModeInRead', 'CORRECTED')\
    .set('spark.sql.parquet.enableVectorizedReader', 'false')\
    .set('spark.kryoserializer.buffer.maxValue', '2044018')\
    .set('spark.sql.autoBroadcastJoinThreshold', -1)\
    .set('spark.sql.broadcastTimeout', -1)\
    .set("spark.sql.catalogImplementation", "hive")\
    .set("mapreduce.input.fileinputformat.input.dir.recursive", 'true')\
    .set('spark.hive.mapred.supports.subdirectories', 'true')\
    .set('spark.sql.hive.convertMetastoreParquet', 'true')

In [7]:
ss = SparkSession \
    .builder \
    .appName(app_name) \
    .config(conf=conf) \
    .getOrCreate()
ss

Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
25/05/20 12:39:57 WARN Utils: Service 'SparkUI' could not bind on port 4040. Attempting port 4041.
25/05/20 12:40:02 WARN HiveConf: HiveConf of name hive.mapred.supports.subdirectories does not exist
25/05/20 12:40:02 WARN Utils: spark.executor.instances less than spark.dynamicAllocation.minExecutors is invalid, ignoring its setting, please update your configs.
25/05/20 12:40:03 WARN Client: Exception encountered while connecting to the server 
org.apache.hadoop.ipc.RemoteException(org.apache.hadoop.ipc.StandbyException): Operation category READ is not supported in state standby. Visit https://s.apache.org/sbnn-error
	at org.apache.hadoop.security.SaslRpcClient.saslConnect(SaslRpcClient.java:376)
	at org.apache.hadoop.ipc.Client$Connection.setupSaslConnection(Client.java:623)
	at org.apache.hadoop.ipc.Client$Connection.access$2300(Client.java:414)
	at org.ap

In [8]:
sc = ss.sparkContext
sc.setLogLevel('ERROR')

### Чтение данных

In [ ]:
df = ss.read.parquet('hdfs://arnsdpldbr2/user/team/team_uim_crm/18281950/chess_merged/all_data_18_22')
df.show(20,truncate=False)

In [12]:
df.columns

['Link',
 'Date',
 'Round',
 'White',
 'Black',
 'Result',
 'Tournament',
 'CurrentPosition',
 'ECO',
 'ECOUrl',
 'WhiteElo',
 'BlackElo',
 'TimeControl',
 'Termination',
 'StartTime',
 'EndDate',
 'EndTime',
 'Moves',
 'Moves_length',
 'Tournament_long',
 'Moves_clean',
 'sf_n1_d22',
 'sf_n10_d18']

### Перевод шахматной нотации к единой

In [10]:
import chess

def change_notation(moves):

    board = chess.Board()
    stockfish_moves = []

    for move in moves:
        try:
            board.push_san(move)
            last_move_uci = board.peek().uci()
            stockfish_moves.append(last_move_uci)
        except:
            return None
    return stockfish_moves

In [11]:
change_notation_udf = udf(change_notation, ArrayType(StringType()))
df = df.withColumn('Moves_sf', change_notation_udf(df['Moves_clean']))

In [12]:
df.show(truncate=False)

+-----------+----------+-----+--------------------+---------------+-------+----------+--------------------------------------------------------+---+-----------------------------------------------------------------------------+--------+--------+-----------+-----------------------------------------+---------+----------+--------+-------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------

Игры, в которых отсутствуют ходы - удаляются

In [13]:
df = df.filter( 'Moves_sf is not null')

### Разделение на дебют, мительшпиль и эндшпиль - части игры

In [15]:
def moves_separation(mov_len):
    deb_start = 0
    mit_start = min(10, mov_len)
    part = (mov_len - 10)//5
    mittel = 3*part
    end_start = mit_start + mittel
    if mit_start % 2:
        mit_start = mit_start - 1
    if end_start % 2:
        end_start = end_start - 1
    return mit_start, end_start, mov_len

In [16]:
moves_separation_udf = udf(moves_separation, ArrayType(StringType()))
df = df.withColumn('parts_idxs', moves_separation_udf(df['Moves_length']))

In [17]:
df.show(10)

+-----------+----------+-----+--------------------+-----------+-------+----------+--------------------+---+--------------------+--------+--------+-----------+------------------+---------+----------+--------+--------------------+------------+---------------+--------------------+--------------------+--------------------+--------------------+-------------+
|       Link|      Date|Round|               White|      Black| Result|Tournament|     CurrentPosition|ECO|              ECOUrl|WhiteElo|BlackElo|TimeControl|       Termination|StartTime|   EndDate| EndTime|               Moves|Moves_length|Tournament_long|         Moves_clean|           sf_n1_d22|          sf_n10_d18|            Moves_sf|   parts_idxs|
+-----------+----------+-----+--------------------+-----------+-------+----------+--------------------+---+--------------------+--------+--------+-----------+------------------+---------+----------+--------+--------------------+------------+---------------+--------------------+----------

### Подсчет доли совпадений со stockfish на 1 линии для белых и черных в каждой партии

In [18]:
def coincidences_1(moves, sf, parts):
    real = sf 
    pred = moves
    
    coincidences_black_m = 0
    coincidences_white_m = 0
    
    coincidences_black_e = 0
    coincidences_white_e = 0
    
    coincidences_black_d = 0
    coincidences_white_d = 0
    
    if parts[0] == 0:
        return  [coincidences_white_d, coincidences_white_m, coincidences_white_e], [coincidences_black_d, coincidences_black_m, coincidences_black_e]

    for i in range(len(real)):
        if pred[i] == real[i][0]['_1']:
            if not i%2:
                if float(parts[0]) > i:
                    coincidences_white_d += 1
                elif float(parts[1]) > i:
                    coincidences_white_m += 1
                else:
                    coincidences_white_e += 1
            else:
                if float(parts[0]) > i:
                    coincidences_black_d += 1
                elif float(parts[1]) > i:
                    coincidences_black_m += 1
                else:
                    coincidences_black_e += 1
                
    return  [coincidences_white_d, coincidences_white_m, coincidences_white_e], [coincidences_black_d, coincidences_black_m, coincidences_black_e]

In [19]:
coincidences_1_udf = udf(coincidences_1, ArrayType(ArrayType(StringType())))
df = df.withColumn('coin_w_d', coincidences_1_udf(df['Moves_sf'], df['sf_n1_d22'], df['parts_idxs'])[0][0])
df = df.withColumn('coin_w_m', coincidences_1_udf(df['Moves_sf'], df['sf_n1_d22'], df['parts_idxs'])[0][1])
df = df.withColumn('coin_w_e', coincidences_1_udf(df['Moves_sf'], df['sf_n1_d22'], df['parts_idxs'])[0][2])

df = df.withColumn('coin_b_d', coincidences_1_udf(df['Moves_sf'], df['sf_n1_d22'], df['parts_idxs'])[1][0])
df = df.withColumn('coin_b_m', coincidences_1_udf(df['Moves_sf'], df['sf_n1_d22'], df['parts_idxs'])[1][1])
df = df.withColumn('coin_b_e', coincidences_1_udf(df['Moves_sf'], df['sf_n1_d22'], df['parts_idxs'])[1][2])

In [20]:
df.show(10)

+-----------+----------+-----+--------------------+-----------+-------+----------+--------------------+---+--------------------+--------+--------+-----------+------------------+---------+----------+--------+--------------------+------------+---------------+--------------------+--------------------+--------------------+--------------------+-------------+--------+--------+--------+--------+--------+--------+
|       Link|      Date|Round|               White|      Black| Result|Tournament|     CurrentPosition|ECO|              ECOUrl|WhiteElo|BlackElo|TimeControl|       Termination|StartTime|   EndDate| EndTime|               Moves|Moves_length|Tournament_long|         Moves_clean|           sf_n1_d22|          sf_n10_d18|            Moves_sf|   parts_idxs|coin_w_d|coin_w_m|coin_w_e|coin_b_d|coin_b_m|coin_b_e|
+-----------+----------+-----+--------------------+-----------+-------+----------+--------------------+---+--------------------+--------+--------+-----------+------------------+---

In [21]:
import re

def coin_fraction_deb(coins, parts):
        return float(coins)*2 / 10
    
def coin_fraction_mit(coins, parts):
        return float(coins)*2 / (float(parts[1]) - float(parts[0]) + 1)
    
def coin_fraction_end(coins, parts):
        return float(coins)*2 / (float(parts[2]) - float(parts[1]) + 1)
    

In [22]:
coin_fraction_d_udf = udf(coin_fraction_deb, FloatType())
coin_fraction_m_udf = udf(coin_fraction_mit, FloatType())
coin_fraction_e_udf = udf(coin_fraction_end, FloatType())

df = df.withColumn('coin_w_frac_d', coin_fraction_d_udf(df['coin_w_d'], df['parts_idxs']))
df = df.withColumn('coin_w_frac_m', coin_fraction_m_udf(df['coin_w_m'], df['parts_idxs']))
df = df.withColumn('coin_w_frac_e', coin_fraction_e_udf(df['coin_w_e'], df['parts_idxs']))

df = df.withColumn('coin_b_frac_d', coin_fraction_d_udf(df['coin_b_d'], df['parts_idxs']))
df = df.withColumn('coin_b_frac_m', coin_fraction_m_udf(df['coin_b_m'], df['parts_idxs']))
df = df.withColumn('coin_b_frac_e', coin_fraction_e_udf(df['coin_b_e'], df['parts_idxs']))

In [23]:
df.show(10)

+-----------+----------+-----+--------------------+-----------+-------+----------+--------------------+---+--------------------+--------+--------+-----------+------------------+---------+----------+--------+--------------------+------------+---------------+--------------------+--------------------+--------------------+--------------------+-------------+--------+--------+--------+--------+--------+--------+-------------+-------------+-------------+-------------+-------------+-------------+
|       Link|      Date|Round|               White|      Black| Result|Tournament|     CurrentPosition|ECO|              ECOUrl|WhiteElo|BlackElo|TimeControl|       Termination|StartTime|   EndDate| EndTime|               Moves|Moves_length|Tournament_long|         Moves_clean|           sf_n1_d22|          sf_n10_d18|            Moves_sf|   parts_idxs|coin_w_d|coin_w_m|coin_w_e|coin_b_d|coin_b_m|coin_b_e|coin_w_frac_d|coin_w_frac_m|coin_w_frac_e|coin_b_frac_d|coin_b_frac_m|coin_b_frac_e|
+-----------

### Подсчет доли совпадений для 18 линии

In [ ]:
df = df.withColumn('coin_w_d_18', coincidences_1_udf(df['Moves_sf'], df['sf_n10_d18'], df['parts_idxs'])[0][0])
df = df.withColumn('coin_w_m_18', coincidences_1_udf(df['Moves_sf'], df['sf_n10_d18'], df['parts_idxs'])[0][1])
df = df.withColumn('coin_w_e_18', coincidences_1_udf(df['Moves_sf'], df['sf_n10_d18'], df['parts_idxs'])[0][2])

df = df.withColumn('coin_b_d_18', coincidences_1_udf(df['Moves_sf'], df['sf_n10_d18'], df['parts_idxs'])[1][0])
df = df.withColumn('coin_b_m_18', coincidences_1_udf(df['Moves_sf'], df['sf_n10_d18'], df['parts_idxs'])[1][1])
df = df.withColumn('coin_b_e_18', coincidences_1_udf(df['Moves_sf'], df['sf_n10_d18'], df['parts_idxs'])[1][2])

In [ ]:
df = df.withColumn('coin_w_frac_d_18', coin_fraction_d_udf(df['coin_w_d_18'], df['parts_idxs']))
df = df.withColumn('coin_w_frac_m_18', coin_fraction_m_udf(df['coin_w_m_18'], df['parts_idxs']))
df = df.withColumn('coin_w_frac_e_18', coin_fraction_e_udf(df['coin_w_e_18'], df['parts_idxs']))

df = df.withColumn('coin_b_frac_d_18', coin_fraction_d_udf(df['coin_b_d_18'], df['parts_idxs']))
df = df.withColumn('coin_b_frac_m_18', coin_fraction_m_udf(df['coin_b_m_18'], df['parts_idxs']))
df = df.withColumn('coin_b_frac_e_18', coin_fraction_e_udf(df['coin_b_e_18'], df['parts_idxs']))

In [ ]:
df.show(10)

+-----------+----------+-----+-------------------+--------------+-------+----------+--------------------+---+--------------------+--------+--------+-----------+--------------------+---------+----------+--------+--------------------+------------+---------------+--------------------+--------------------+--------------------+--------------------+-------------+--------+--------+--------+--------+--------+--------+-------------+-------------+-------------+-------------+-------------+-------------+-----------+-----------+-----------+-----------+-----------+-----------+----------------+----------------+----------------+----------------+----------------+----------------+
|       Link|      Date|Round|              White|         Black| Result|Tournament|     CurrentPosition|ECO|              ECOUrl|WhiteElo|BlackElo|TimeControl|         Termination|StartTime|   EndDate| EndTime|               Moves|Moves_length|Tournament_long|         Moves_clean|           sf_n1_d22|          sf_n10_d18|    

### Время хода и агрегирование

In [61]:


def motion_times(timeControl, moves, parts):
    all_time_w = timeControl
    inc = 0
    
    if len(all_time_w) > 3:
        inc = float(all_time_w[4])
        all_time_w = float(all_time_w[0:3])    
    else:
        all_time_w = float(all_time_w)
        
    all_time_b = all_time_w
    time_pattern = r'\d+:\d+:\d+\.?\d?'
    times = re.findall(time_pattern, moves)
    times_white_m, times_white_e, times_white_d, times_black_d, times_black_m, times_black_e = [], [], [], [], [], []
    
    for i, time in enumerate(times):
        hours, minutes, seconds = time.split(':')
        now_time = float(hours)*3600 + float(minutes)*60 + float(seconds) 
        if not i%2:
            all_time_w += inc
            if float(parts[0]) > i:
                times_white_d.append(all_time_w - now_time)
            elif float(parts[1]) > i:
                times_white_m.append(all_time_w - now_time)
                
            else:
                times_white_e.append(all_time_w - now_time)
            all_time_w = now_time 
        else: 
            all_time_b += inc
            if float(parts[0]) > i:
                times_black_d.append(all_time_b - now_time)
            elif float(parts[1]) > i:
                times_black_m.append(all_time_b - now_time)
            else:
                times_black_e.append(all_time_b - now_time)
            all_time_b = now_time

    return [times_white_d, times_white_m, times_white_e, times_black_d, times_black_m, times_black_e]

In [ ]:
motion_times_udf = udf(motion_times, ArrayType(ArrayType(FloatType())))
df = df.withColumn('Moves_time', motion_times_udf(df['TimeControl'], df['Moves'], df['parts_idxs']))

In [ ]:
df.show(10)

+-----------+----------+-----+----------------+--------------------+-------+--------------+--------------------+---+--------------------+--------+--------+-----------+-------------------+---------+----------+--------+--------------------+------------+--------------------+--------------------+--------------------+--------------------+--------------------+-------------+--------+--------+--------+--------+--------+--------+-------------+-------------+-------------+-------------+-------------+-------------+-----------+-----------+-----------+-----------+-----------+-----------+----------------+----------------+----------------+----------------+----------------+----------------+--------------------+
|       Link|      Date|Round|           White|               Black| Result|    Tournament|     CurrentPosition|ECO|              ECOUrl|WhiteElo|BlackElo|TimeControl|        Termination|StartTime|   EndDate| EndTime|               Moves|Moves_length|     Tournament_long|         Moves_clean|   

### Массив совпадений (для дальнейших подсчетов)

In [65]:
def coincidences_array_(moves, sf):

    coincidences_array = []

    for i in range(min(len(moves), len(sf))):
        if moves[i] == sf[i][0]['_1']:
            coincidences_array.append(1)
        else: 
            coincidences_array.append(0)
            
    return coincidences_array

In [ ]:
coincidences_array_udf = udf(coincidences_array_, ArrayType(IntegerType()))
df = df.withColumn('coin_array', coincidences_array_udf(df['Moves_sf'], df['sf_n1_d22']))
df = df.withColumn('coin_array_18', coincidences_array_udf(df['Moves_sf'], df['sf_n10_d18']))

In [ ]:
df.show(10)

+-----------+----------+-----+--------------+----------------+-------+----------+--------------------+---+--------------------+--------+--------+-----------+-------------------+---------+----------+--------+--------------------+------------+---------------+--------------------+--------------------+--------------------+--------------------+------------+--------+--------+--------+--------+--------+--------+-------------+-------------+-------------+-------------+-------------+-------------+-----------+-----------+-----------+-----------+-----------+-----------+----------------+----------------+----------------+----------------+----------------+----------------+--------------------+--------------------+--------------------+
|       Link|      Date|Round|         White|           Black| Result|Tournament|     CurrentPosition|ECO|              ECOUrl|WhiteElo|BlackElo|TimeControl|        Termination|StartTime|   EndDate| EndTime|               Moves|Moves_length|Tournament_long|         Move

### Время хода при совпадении и несовпадении с ходами stockfish

In [68]:
def coincidences_time(coin_arr, times, parts):
    
    times_w = times[0] + times[1] + times[2]
    times_b = times[3] + times[4] + times[5]
    
    coincidences_time_w_d, another_time_w_d = [], []
    coincidences_time_b_d, another_time_b_d = [], []
    
    coincidences_time_w_m, another_time_w_m = [], []
    coincidences_time_b_m, another_time_b_m = [], []
    
    coincidences_time_w_e, another_time_w_e = [], []
    coincidences_time_b_e, another_time_b_e = [], []
    
    counter_white, counter_black = 0, 0
    
    for i, res in enumerate(coin_arr):
        if i < float(parts[0]):
            if res:
                if not i%2:
                    coincidences_time_w_d.append(float(times_w[counter_white]))
                    counter_white += 1
                else:
                    coincidences_time_b_d.append(float(times_b[counter_black]))
                    counter_black += 1
            else:
                if not i%2:
                    another_time_w_d.append(float(times_w[counter_white]))
                    counter_white += 1
                else:
                    another_time_b_d.append(float(times_b[counter_black]))
                    counter_black += 1
        elif i < float(parts[1]):
            if res:
                if not i%2:
                    coincidences_time_w_m.append(float(times_w[counter_white]))
                    counter_white += 1
                else:
                    coincidences_time_b_m.append(float(times_b[counter_black]))
                    counter_black += 1
            else:
                if not i%2:
                    another_time_w_m.append(float(times_w[counter_white]))
                    counter_white += 1
                else:
                    another_time_b_m.append(float(times_b[counter_black]))
                    counter_black += 1
        else:
            if res:
                if not i%2:
                    coincidences_time_w_e.append(float(times_w[counter_white]))
                    counter_white += 1
                else:
                    coincidences_time_b_e.append(float(times_b[counter_black]))
                    counter_black += 1
            else:
                if not i%2:
                    another_time_w_e.append(float(times_w[counter_white]))
                    counter_white += 1
                else:
                    another_time_b_e.append(float(times_b[counter_black]))
                    counter_black += 1
            
    
                
    mean_coin_time_w_d = sum(coincidences_time_w_d)/max(len(coincidences_time_w_d), 1)
    mean_another_time_w_d = sum(another_time_w_d)/max(len(another_time_w_d), 1)
    
    mean_coin_time_w_m = sum(coincidences_time_w_m)/max(len(coincidences_time_w_m), 1)
    mean_another_time_w_m = sum(another_time_w_m)/max(len(another_time_w_m), 1)
    
    mean_coin_time_w_e = sum(coincidences_time_w_e)/max(len(coincidences_time_w_e), 1)
    mean_another_time_w_e = sum(another_time_w_e)/max(len(another_time_w_e), 1)
    
    mean_coin_time_b_d = sum(coincidences_time_b_d)/max(len(coincidences_time_b_d), 1)
    mean_another_time_b_d = sum(another_time_b_d)/max(len(another_time_b_d), 1)
    
    mean_coin_time_b_m = sum(coincidences_time_b_m)/max(len(coincidences_time_b_m), 1)
    mean_another_time_b_m = sum(another_time_b_m)/max(len(another_time_b_m), 1)
    
    mean_coin_time_b_e = sum(coincidences_time_b_e)/max(len(coincidences_time_b_e), 1)
    mean_another_time_b_e = sum(another_time_b_e)/max(len(another_time_b_e), 1)

    return mean_coin_time_w_d, mean_another_time_w_d, mean_coin_time_w_m, mean_another_time_w_m, mean_coin_time_w_e,\
        mean_another_time_w_e, mean_coin_time_b_d, mean_another_time_b_d, mean_coin_time_b_m, \
        mean_another_time_b_m, mean_coin_time_b_e, mean_another_time_b_e

In [ ]:
coincidences_time_udf = udf(coincidences_time, ArrayType(FloatType()))
df = df.withColumn('mean_times', coincidences_time_udf(df['coin_array'], df['Moves_time'], df['parts_idxs']))


In [ ]:
df.show(10)

+-----------+----------+-----+-----------------+---------------+-------+--------------+--------------------+---+--------------------+--------+--------+-----------+--------------------+---------+----------+--------+--------------------+------------+--------------------+--------------------+--------------------+--------------------+--------------------+-------------+--------+--------+--------+--------+--------+--------+-------------+-------------+-------------+-------------+-------------+-------------+-----------+-----------+-----------+-----------+-----------+-----------+----------------+----------------+----------------+----------------+----------------+----------------+--------------------+--------------------+--------------------+--------------------+
|       Link|      Date|Round|            White|          Black| Result|    Tournament|     CurrentPosition|ECO|              ECOUrl|WhiteElo|BlackElo|TimeControl|         Termination|StartTime|   EndDate| EndTime|               Moves|M

### Победа/ ничья

In [71]:
def spliter(res):
    if res == '1-0':
        return 1.0,0.0
    elif res == '0-1':
        return 0.0, 1.0
    else:
        return 0.5, 0.5

In [ ]:
spliter_udf = udf(spliter, ArrayType(FloatType()))
df = df.withColumn('Result_w', spliter_udf(df['Result'])[0])
df = df.withColumn('Result_b', spliter_udf(df['Result'])[1])

In [ ]:
df.show(5)

+-----------+----------+-----+------------------+------------+-------+----------+--------------------+---+--------------------+--------+--------+-----------+--------------------+---------+----------+--------+--------------------+------------+---------------+--------------------+--------------------+--------------------+--------------------+-------------+--------+--------+--------+--------+--------+--------+-------------+-------------+-------------+-------------+-------------+-------------+-----------+-----------+-----------+-----------+-----------+-----------+----------------+----------------+----------------+----------------+----------------+----------------+--------------------+--------------------+--------------------+--------------------+--------+--------+
|       Link|      Date|Round|             White|       Black| Result|Tournament|     CurrentPosition|ECO|              ECOUrl|WhiteElo|BlackElo|TimeControl|         Termination|StartTime|   EndDate| EndTime|               Moves|

### Принадлежность партии к турнирам на деньги

In [80]:
def tournament(tournament):
    return int(tournament == 'titled-tuesday')

In [ ]:
tournament_udf = udf(tournament, IntegerType())
df = df.withColumn('Tournament_is_tt', tournament_udf(df['Tournament']))

In [ ]:
df.show(10)

+-----------+----------+-----+---------------+-----------------+------+----------+--------------------+---+--------------------+--------+--------+-----------+------------------+---------+----------+--------+--------------------+------------+---------------+--------------------+--------------------+--------------------+--------------------+-------------+--------+--------+--------+--------+--------+--------+-------------+-------------+-------------+-------------+-------------+-------------+-----------+-----------+-----------+-----------+-----------+-----------+----------------+----------------+----------------+----------------+----------------+----------------+--------------------+--------------------+--------------------+--------------------+--------+--------+----------------+
|       Link|      Date|Round|          White|            Black|Result|Tournament|     CurrentPosition|ECO|              ECOUrl|WhiteElo|BlackElo|TimeControl|       Termination|StartTime|   EndDate| EndTime|      

### Соотношение рейтингов игрока и его соперника

In [83]:
def rate_ratio(whiteELO, blackELO):
    return float(whiteELO)/float(blackELO)

In [ ]:
rate_ratio_udf = udf(rate_ratio, FloatType())
df = df.withColumn('rate_ratio', rate_ratio_udf(df['WhiteELO'], df['BlackELO']))

In [ ]:
df.show(20)

+-----------+----------+-----+------------------+---------------+-------+--------------+--------------------+---+--------------------+--------+--------+-----------+------------------+---------+----------+--------+--------------------+------------+--------------------+--------------------+--------------------+--------------------+--------------------+-------------+--------+--------+--------+--------+--------+--------+-------------+-------------+-------------+-------------+-------------+-------------+-----------+-----------+-----------+-----------+-----------+-----------+----------------+----------------+----------------+----------------+----------------+----------------+--------------------+--------------------+--------------------+--------------------+--------+--------+----------------+----------+
|       Link|      Date|Round|             White|          Black| Result|    Tournament|     CurrentPosition|ECO|              ECOUrl|WhiteElo|BlackElo|TimeControl|       Termination|StartTim

### Вероятность победы

In [86]:
def rate_prob(whiteELO, blackELO):
    D = float(whiteELO) - float(blackELO)
    P = 1/ (1 + 10**(-D/2000))
    return P

In [ ]:
rate_prob_udf = udf(rate_prob, FloatType())
df = df.withColumn('rate_prob', rate_prob_udf(df['WhiteELO'], df['BlackELO']))

In [ ]:
df.show(20)

+-----------+----------+-----+--------------------+--------------------+-------+----------+--------------------+---+--------------------+--------+--------+-----------+-------------------+---------+----------+--------+--------------------+------------+---------------+--------------------+--------------------+--------------------+--------------------+-------------+--------+--------+--------+--------+--------+--------+-------------+-------------+-------------+-------------+-------------+-------------+-----------+-----------+-----------+-----------+-----------+-----------+----------------+----------------+----------------+----------------+----------------+----------------+--------------------+--------------------+--------------------+--------------------+--------+--------+----------------+----------+----------+
|       Link|      Date|Round|               White|               Black| Result|Tournament|     CurrentPosition|ECO|              ECOUrl|WhiteElo|BlackElo|TimeControl|        Termin

### Статистики за последний год игр

In [ ]:
df_unpivoted = df.select(
    col('Link'),
    col('Date'),
    col('White').alias('player'),
    col('coin_w_frac_d').alias('scores_d'),
    col('coin_w_frac_m').alias('scores_m'),
    col('coin_w_frac_e').alias('scores_e'),
    col('coin_w_frac_d_18').alias('scores_d_18'),
    col('coin_w_frac_m_18').alias('scores_m_18'),
    col('coin_w_frac_e_18').alias('scores_e_18'),
    col('StartTime')
).union(
    df.select(
    col('Link'),
    col('Date'),
    col('Black').alias('player'),
    col('coin_b_frac_d').alias('scores_d'),
    col('coin_b_frac_m').alias('scores_m'),
    col('coin_b_frac_e').alias('scores_e'),
    col('coin_b_frac_d_18').alias('scores_d_18'),
    col('coin_b_frac_m_18').alias('scores_m_18'),
    col('coin_b_frac_e_18').alias('scores_e_18'),
    col('StartTime')
)
)

df_unpivoted.show()

+-----------+----------+--------------------+--------+----------+----------+-----------+-----------+-----------+---------+
|       Link|      Date|              player|scores_d|  scores_m|  scores_e|scores_d_18|scores_m_18|scores_e_18|StartTime|
+-----------+----------+--------------------+--------+----------+----------+-----------+-----------+-----------+---------+
|11812484495|2021.04.10|          ChessDawgs|     0.6|0.42105263|0.19512194|        0.6|  0.3859649| 0.24390244| 16:53:07|
|13839098867|2021.05.04|              najaro|     0.6|0.53333336|0.30769232|        0.8|  0.6666667| 0.30769232| 03:47:34|
|16311401759|2021.06.01|Gudmundur_Kjartan...|     0.4| 0.5185185|0.54545456|        0.4|  0.5185185| 0.54545456| 18:36:30|
|16312438699|2021.06.01|           PotapovaM|     0.4|0.26666668|0.44444445|        0.2| 0.35555556|  0.3888889| 18:47:30|
|18986264797|2021.07.02|       LazarusIcarus|     0.6|0.46575344|0.53061223|        0.4| 0.43835616|  0.6122449| 17:38:32|
|24673929959|202

In [ ]:
df.filter(col('Link') == '86996929945').show(10)

+-----------+----------+-----+---------+------------+------+----------+--------------------+---+--------------------+--------+--------+-----------+------------------+---------+----------+--------+--------------------+------------+---------------+--------------------+--------------------+--------------------+--------------------+------------+--------+--------+--------+--------+--------+--------+-------------+-------------+-------------+-------------+-------------+-------------+-----------+-----------+-----------+-----------+-----------+-----------+----------------+----------------+----------------+----------------+----------------+----------------+--------------------+--------------------+--------------------+--------------------+--------+--------+----------------+----------+----------+
|       Link|      Date|Round|    White|       Black|Result|Tournament|     CurrentPosition|ECO|              ECOUrl|WhiteElo|BlackElo|TimeControl|       Termination|StartTime|   EndDate| EndTime|       

In [67]:
df_unpivoted.filter(col('Link') == '86996929945').show(10)

+-----------+----------+------------+--------+----------+--------+-----------+-----------+-----------+---------+
|       Link|      Date|      player|scores_d|  scores_m|scores_e|scores_d_18|scores_m_18|scores_e_18|StartTime|
+-----------+----------+------------+--------+----------+--------+-----------+-----------+-----------+---------+
|86996929945|2023.08.28|   Timkolov2|     0.8|0.84210527|   0.625|        0.6| 0.84210527|        0.5| 21:23:22|
|86996929945|2023.08.28|dsanchez1973|     0.8|0.84210527|    0.75|        0.6| 0.84210527|      0.625| 21:23:22|
+-----------+----------+------------+--------+----------+--------+-----------+-----------+-----------+---------+



In [97]:
from pyspark.sql import Window

window = Window.partitionBy('player').orderBy('Date')

prev_scores_d = F.collect_list(col('scores_d')).over(window.rowsBetween(Window.unboundedPreceding, -1))
prev_scores_m = F.collect_list(col('scores_m')).over(window.rowsBetween(Window.unboundedPreceding, -1))
prev_scores_e = F.collect_list(col('scores_e')).over(window.rowsBetween(Window.unboundedPreceding, -1))
prev_scores_d_18 = F.collect_list(col('scores_d_18')).over(window.rowsBetween(Window.unboundedPreceding, -1))
prev_scores_m_18 = F.collect_list(col('scores_m_18')).over(window.rowsBetween(Window.unboundedPreceding, -1))
prev_scores_e_18 = F.collect_list(col('scores_e_18')).over(window.rowsBetween(Window.unboundedPreceding, -1))

                           
df_with_prev = df_unpivoted.withColumn('prev_scores_d', prev_scores_d)
df_with_prev = df_with_prev.withColumn('prev_scores_m', prev_scores_m)
df_with_prev = df_with_prev.withColumn('prev_scores_e', prev_scores_e)
df_with_prev = df_with_prev.withColumn('prev_scores_d_18', prev_scores_d_18)
df_with_prev = df_with_prev.withColumn('prev_scores_m_18', prev_scores_m_18)
df_with_prev = df_with_prev.withColumn('prev_scores_e_18', prev_scores_e_18)

                           
df_with_prev.show(100)

+-----------+----------+----------------+--------+----------+----------+-----------+-----------+-----------+---------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+
|       Link|      Date|          player|scores_d|  scores_m|  scores_e|scores_d_18|scores_m_18|scores_e_18|StartTime|       prev_scores_d|       prev_scores_m|       prev_scores_e|    prev_scores_d_18|    prev_scores_m_18|    prev_scores_e_18|
+-----------+----------+----------------+--------+----------+----------+-----------+-----------+-----------+---------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+
|33868250575|2021.12.21|        12lemhaf|     0.4| 0.3809524|0.44444445|        0.6|  0.3809524| 0.26666668| 23:31:29|                  []|                  []|                  []|                  []|                  []|                  []|
|34491032799|2021.12

In [ ]:
df_join_white = df.alias('df_').join(df_with_prev.alias('prev'),
                             (col("df_.White") == col("prev.player")) & (col("df_.Link") == col("prev.Link")),
                             'left') \
                        .select(df['*'], col('prev.prev_scores_d').alias('prev_scores_d_w'), 
                               col('prev.prev_scores_m').alias('prev_scores_m_w'),
                               col('prev.prev_scores_e').alias('prev_scores_e_w'),
                               col('prev.prev_scores_d_18').alias('prev_scores_d_18_w'),
                               col('prev.prev_scores_m_18').alias('prev_scores_m_18_w'),
                               col('prev.prev_scores_e_18').alias('prev_scores_e_18_w'))



In [99]:
df_join_black = df_join_white.alias('d_f').join(df_with_prev.alias('prev'),
                             (col('d_f.Black') == col("prev.player")) & (col("d_f.Link") == col("prev.Link")),
                             'left') \
                        .select(df_join_white['*'], col('prev.prev_scores_d').alias('prev_scores_d_b'), 
                               col('prev.prev_scores_m').alias('prev_scores_m_b'),
                               col('prev.prev_scores_e').alias('prev_scores_e_b'),
                               col('prev.prev_scores_d_18').alias('prev_scores_d_18_b'),
                               col('prev.prev_scores_m_18').alias('prev_scores_m_18_b'),
                               col('prev.prev_scores_e_18').alias('prev_scores_e_18_b'))

In [100]:
df_join_black.show(100)  

+-----------+----------+-----+------------------+------------------+-------+--------------+--------------------+---+--------------------+--------+--------+-----------+--------------------+---------+----------+--------+--------------------+------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------+--------+--------+--------+--------+--------+--------+-------------+-------------+-------------+-------------+-------------+-------------+-----------+-----------+-----------+-----------+-----------+-----------+----------------+----------------+----------------+----------------+----------------+----------------+--------------------+--------------------+--------------------+--------------------+--------+--------+----------------+----------+----------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+-------

In [103]:
def mean_scores(prev):
    if len(prev) == 0:
        return -1
    return sum(prev)/len(prev)

In [104]:
mean_scores_udf = udf(mean_scores, FloatType())
df_join_black = df_join_black.withColumn('mean_prev_scores_d_w', mean_scores_udf(df_join_black['prev_scores_d_w']))
df_join_black = df_join_black.withColumn('mean_prev_scores_m_w', mean_scores_udf(df_join_black['prev_scores_m_w']))
df_join_black = df_join_black.withColumn('mean_prev_scores_e_w', mean_scores_udf(df_join_black['prev_scores_e_w']))
df_join_black = df_join_black.withColumn('mean_prev_scores_d_18_w', mean_scores_udf(df_join_black['prev_scores_d_18_w']))
df_join_black = df_join_black.withColumn('mean_prev_scores_m_18_w', mean_scores_udf(df_join_black['prev_scores_m_18_w']))
df_join_black = df_join_black.withColumn('mean_prev_scores_e_18_w', mean_scores_udf(df_join_black['prev_scores_e_18_w']))

df_join_black = df_join_black.withColumn('mean_prev_scores_d_b', mean_scores_udf(df_join_black['prev_scores_d_b']))
df_join_black = df_join_black.withColumn('mean_prev_scores_m_b', mean_scores_udf(df_join_black['prev_scores_m_b']))
df_join_black = df_join_black.withColumn('mean_prev_scores_e_b', mean_scores_udf(df_join_black['prev_scores_e_b']))
df_join_black = df_join_black.withColumn('mean_prev_scores_d_18_b', mean_scores_udf(df_join_black['prev_scores_d_18_b']))
df_join_black = df_join_black.withColumn('mean_prev_scores_m_18_b', mean_scores_udf(df_join_black['prev_scores_m_18_b']))
df_join_black = df_join_black.withColumn('mean_prev_scores_e_18_b', mean_scores_udf(df_join_black['prev_scores_e_18_b']))



In [105]:
df_join_black.show()

+-----------+----------+-----+------------------+-----------------+------+--------------+--------------------+---+--------------------+--------+--------+-----------+--------------------+---------+----------+--------+--------------------+------------+--------------------+--------------------+--------------------+--------------------+--------------------+-------------+--------+--------+--------+--------+--------+--------+-------------+-------------+-------------+-------------+-------------+-------------+-----------+-----------+-----------+-----------+-----------+-----------+----------------+----------------+----------------+----------------+----------------+----------------+--------------------+--------------------+--------------------+--------------------+--------+--------+----------------+----------+----------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+----------

In [106]:
df_join_black = df_join_black.drop('prev_scores_e_18_b', 'prev_scores_m_18_b', 'prev_scores_d_18_b', 'prev_scores_e_b', 'prev_scores_m_b', 'prev_scores_d_b')
df_join_black = df_join_black.drop('prev_scores_e_18_w', 'prev_scores_m_18_w', 'prev_scores_d_18_w', 'prev_scores_e_w', 'prev_scores_m_w', 'prev_scores_d_w')

In [107]:
df_join_black.show()

+-----------+----------+-----+------------------+-----------------+------+--------------+--------------------+---+--------------------+--------+--------+-----------+--------------------+---------+----------+--------+--------------------+------------+--------------------+--------------------+--------------------+--------------------+--------------------+-------------+--------+--------+--------+--------+--------+--------+-------------+-------------+-------------+-------------+-------------+-------------+-----------+-----------+-----------+-----------+-----------+-----------+----------------+----------------+----------------+----------------+----------------+----------------+--------------------+--------------------+--------------------+--------------------+--------+--------+----------------+----------+----------+--------------------+--------------------+--------------------+-----------------------+-----------------------+-----------------------+--------------------+--------------------+-

In [108]:
df_join_black = df_join_black.fillna({'mean_prev_scores_e_18_b' : -1,
                    'mean_prev_scores_m_18_b' : -1,
                    'mean_prev_scores_d_18_b' : -1,
                    'mean_prev_scores_e_18_w' : -1,
                    'mean_prev_scores_m_18_w' : -1,
                    'mean_prev_scores_d_18_w' : -1,
                    'mean_prev_scores_e_b' : -1,
                    'mean_prev_scores_d_b' : -1,
                    'mean_prev_scores_m_b' : -1,
                    'mean_prev_scores_e_w' : -1,
                    'mean_prev_scores_m_w' : -1,
                    'mean_prev_scores_d_w' : -1})

In [109]:
df_join_black.show()

+-----------+----------+-----+------------------+-----------------+------+--------------+--------------------+---+--------------------+--------+--------+-----------+--------------------+---------+----------+--------+--------------------+------------+--------------------+--------------------+--------------------+--------------------+--------------------+-------------+--------+--------+--------+--------+--------+--------+-------------+-------------+-------------+-------------+-------------+-------------+-----------+-----------+-----------+-----------+-----------+-----------+----------------+----------------+----------------+----------------+----------------+----------------+--------------------+--------------------+--------------------+--------------------+--------+--------+----------------+----------+----------+--------------------+--------------------+--------------------+-----------------------+-----------------------+-----------------------+--------------------+--------------------+-

In [113]:
for i in range(12): 
    df = df.withColumn(f'mean_times_{i}', col('mean_times').getItem(i))

In [114]:
df.show()

+-----------+----------+-----+------------------+-------------+-------+--------------+--------------------+---+--------------------+--------+--------+-----------+--------------------+---------+----------+--------+--------------------+------------+--------------------+--------------------+--------------------+--------------------+--------------------+-------------+--------+--------+--------+--------+--------+--------+-------------+-------------+-------------+-------------+-------------+-------------+-----------+-----------+-----------+-----------+-----------+-----------+----------------+----------------+----------------+----------------+----------------+----------------+--------------------+--------------------+--------------------+--------------------+--------+--------+----------------+----------+----------+--------------------+--------------------+--------------------+-----------------------+-----------------------+-----------------------+--------------------+--------------------+----

### Время за прошедшие игры

In [116]:
df_unpivoted = df.select(
    col('Link'),
    col('Date'),
    col('White').alias('player'),
    col('mean_times_0').alias('mean_time_c_d'),
    col('mean_times_1').alias('mean_time_a_d'),
    col('mean_times_2').alias('mean_time_c_m'),
    col('mean_times_3').alias('mean_time_a_m'),
    col('mean_times_4').alias('mean_time_c_e'),
    col('mean_times_5').alias('mean_time_a_e'),
    col('StartTime')
).union(
    df.select(
    col('Link'),
    col('Date'),
    col('Black').alias('player'),
    col('mean_times_6').alias('mean_time_c_d'),
    col('mean_times_7').alias('mean_time_a_d'),
    col('mean_times_8').alias('mean_time_c_m'),
    col('mean_times_9').alias('mean_time_a_m'),
    col('mean_times_10').alias('mean_time_c_e'),
    col('mean_times_11').alias('mean_time_a_e'),
    col('StartTime')
)
)

df_unpivoted.show()

+-----------+----------+------------------+-------------+-------------+-------------+-------------+-------------+-------------+---------+
|       Link|      Date|            player|mean_time_c_d|mean_time_a_d|mean_time_c_m|mean_time_a_m|mean_time_c_e|mean_time_a_e|StartTime|
+-----------+----------+------------------+-------------+-------------+-------------+-------------+-------------+-------------+---------+
| 3749296672|2019.06.02|          MNikolov|    0.8666667|   0.65000004|    1.9300001|         6.45|     4.383333|        7.025| 10:32:09|
| 3967851774|2019.08.21|  Tigranpatagonian|        1.025|          0.1|         6.65|          4.4|    1.8666667|          5.8| 23:29:33|
|37121388653|2022.01.28|     Turkchess1905|         0.52|          0.0|    1.2545455|         4.05|         1.44|    2.1399999| 15:03:32|
|47470211159|2022.05.28|  Y0ung_Capablanca|          0.4|   0.35000002|    2.6299999|    3.5684211|    1.0777777|         2.25| 09:46:46|
|67976593287|2023.01.20|         L

In [117]:
window = Window.partitionBy('player').orderBy('Date')

prev_time_cd = F.collect_list(col('mean_time_c_d')).over(window.rowsBetween(Window.unboundedPreceding, -1))
prev_time_ad = F.collect_list(col('mean_time_a_d')).over(window.rowsBetween(Window.unboundedPreceding, -1))
prev_time_cm = F.collect_list(col('mean_time_c_m')).over(window.rowsBetween(Window.unboundedPreceding, -1))
prev_time_am = F.collect_list(col('mean_time_a_m')).over(window.rowsBetween(Window.unboundedPreceding, -1))
prev_time_ce = F.collect_list(col('mean_time_c_e')).over(window.rowsBetween(Window.unboundedPreceding, -1))
prev_time_ae = F.collect_list(col('mean_time_a_e')).over(window.rowsBetween(Window.unboundedPreceding, -1))

                           
df_with_prev = df_unpivoted.withColumn('prev_time_cd', prev_time_cd)
df_with_prev = df_with_prev.withColumn('prev_time_ad', prev_time_ad)
df_with_prev = df_with_prev.withColumn('prev_time_cm', prev_time_cm)
df_with_prev = df_with_prev.withColumn('prev_time_am', prev_time_am)
df_with_prev = df_with_prev.withColumn('prev_time_ce', prev_time_ce)
df_with_prev = df_with_prev.withColumn('prev_time_ae', prev_time_ae)

                           
df_with_prev.show(100)

+-----------+----------+----------------+-------------+-------------+-------------+-------------+-------------+-------------+---------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+
|       Link|      Date|          player|mean_time_c_d|mean_time_a_d|mean_time_c_m|mean_time_a_m|mean_time_c_e|mean_time_a_e|StartTime|        prev_time_cd|        prev_time_ad|        prev_time_cm|        prev_time_am|        prev_time_ce|        prev_time_ae|
+-----------+----------+----------------+-------------+-------------+-------------+-------------+-------------+-------------+---------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+
|33868250575|2021.12.21|        12lemhaf|          4.1|          3.1|    4.8333335|    4.7473683|         3.44|         4.05| 23:31:29|                  []|                  []|                  []|                

In [118]:
df_join_white = df.alias('df_').join(df_with_prev.alias('prev'),
                             (col("df_.White") == col("prev.player")) & (col("df_.Link") == col("prev.Link")),
                             'left') \
                        .select(df['*'], col('prev.prev_time_cd').alias('prev_times_cd_w'), 
                               col('prev.prev_time_ad').alias('prev_times_ad_w'),
                               col('prev.prev_time_cm').alias('prev_times_cm_w'),
                               col('prev.prev_time_am').alias('prev_times_am_w'),
                               col('prev.prev_time_ce').alias('prev_times_ce_w'),
                               col('prev.prev_time_ae').alias('prev_times_ae_w'))



In [119]:
df_join_black = df_join_white.alias('d_f').join(df_with_prev.alias('prev'),
                             (col('d_f.Black') == col("prev.player")) & (col("d_f.Link") == col("prev.Link")),
                             'left') \
                        .select(df_join_white['*'], col('prev.prev_time_cd').alias('prev_times_cd_b'), 
                               col('prev.prev_time_ad').alias('prev_times_ad_b'),
                               col('prev.prev_time_cm').alias('prev_times_cm_b'),
                               col('prev.prev_time_am').alias('prev_times_am_b'),
                               col('prev.prev_time_ce').alias('prev_times_ce_b'),
                               col('prev.prev_time_ae').alias('prev_times_ae_b'))

In [120]:
mean_scores_udf = udf(mean_scores, FloatType())
df_join_black = df_join_black.withColumn('mean_prev_times_cd_b', mean_scores_udf(df_join_black['prev_times_cd_b']))
df_join_black = df_join_black.withColumn('mean_prev_times_ad_b', mean_scores_udf(df_join_black['prev_times_ad_b']))
df_join_black = df_join_black.withColumn('mean_prev_times_cm_b', mean_scores_udf(df_join_black['prev_times_cm_b']))
df_join_black = df_join_black.withColumn('mean_prev_times_am_b', mean_scores_udf(df_join_black['prev_times_am_b']))
df_join_black = df_join_black.withColumn('mean_prev_times_ce_b', mean_scores_udf(df_join_black['prev_times_ce_b']))
df_join_black = df_join_black.withColumn('mean_prev_times_ae_b', mean_scores_udf(df_join_black['prev_times_ae_b']))

df_join_black = df_join_black.withColumn('mean_prev_times_cd_w', mean_scores_udf(df_join_black['prev_times_cd_w']))
df_join_black = df_join_black.withColumn('mean_prev_times_ad_w', mean_scores_udf(df_join_black['prev_times_ad_w']))
df_join_black = df_join_black.withColumn('mean_prev_times_cm_w', mean_scores_udf(df_join_black['prev_times_cm_w']))
df_join_black = df_join_black.withColumn('mean_prev_times_am_w', mean_scores_udf(df_join_black['prev_times_am_w']))
df_join_black = df_join_black.withColumn('mean_prev_times_ce_w', mean_scores_udf(df_join_black['prev_times_ce_w']))
df_join_black = df_join_black.withColumn('mean_prev_times_ae_w', mean_scores_udf(df_join_black['prev_times_ae_w']))

In [121]:
df_join_black = df_join_black.drop('prev_times_cd_b', 'prev_times_ad_b', 'prev_times_cm_b', 'prev_times_am_b', 'prev_times_ce_b', 'prev_times_ae_b')
df_join_black = df_join_black.drop('prev_times_cd_w', 'prev_times_ad_w', 'prev_times_cm_w', 'prev_times_am_w', 'prev_times_ce_w', 'prev_times_ae_w')

In [122]:
df_join_black = df_join_black.fillna({'mean_prev_times_cd_b' : -1,
                   'mean_prev_times_ad_b' : -1,
                    'mean_prev_times_cm_b' : -1,
                    'mean_prev_times_am_b' : -1,
                    'mean_prev_times_ce_b' : -1,
                    'mean_prev_times_ae_b' : -1,
                    'mean_prev_times_cd_w' : -1,
                    'mean_prev_times_ad_w' : -1,
                    'mean_prev_times_cm_w' : -1,
                    'mean_prev_times_am_w' : -1,
                    'mean_prev_times_ce_w' : -1,
                    'mean_prev_times_ae_w' : -1})

In [123]:
df_join_black.show()

+-----------+----------+-----+--------------------+-------------+-------+----------+--------------------+---+--------------------+--------+--------+-----------+-------------------+---------+----------+--------+--------------------+------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------+--------+--------+--------+--------+--------+--------+-------------+-------------+-------------+-------------+-------------+-------------+-----------+-----------+-----------+-----------+-----------+-----------+----------------+----------------+----------------+----------------+----------------+----------------+--------------------+--------------------+--------------------+--------------------+--------+--------+----------------+----------+----------+--------------------+--------------------+--------------------+-----------------------+-----------------------+-----------------------+--------------------+--------------------+------

### Окончание партии - encoding

In [114]:
df.show()

+-----------+----------+-----+------------------+-------------+-------+--------------+--------------------+---+--------------------+--------+--------+-----------+--------------------+---------+----------+--------+--------------------+------------+--------------------+--------------------+--------------------+--------------------+--------------------+-------------+--------+--------+--------+--------+--------+--------+-------------+-------------+-------------+-------------+-------------+-------------+-----------+-----------+-----------+-----------+-----------+-----------+----------------+----------------+----------------+----------------+----------------+----------------+--------------------+--------------------+--------------------+--------------------+--------+--------+----------------+----------+----------+--------------------+--------------------+--------------------+-----------------------+-----------------------+-----------------------+--------------------+--------------------+----

In [115]:
df.select('Termination').distinct().show(truncate=False)

+-----------------------------------------+
|Termination                              |
+-----------------------------------------+
|won on time                              |
|drawn by agreement                       |
|drawn by 50-move rule                    |
|drawn by timeout vs insufficient material|
|drawn by repetition                      |
|drawn by insufficient material           |
|won by checkmate                         |
|won - game abandoned                     |
|drawn by stalemate                       |
|won by resignation                       |
+-----------------------------------------+



In [118]:
df = df.withColumn('won on time', F.when(df.Termination == 'won on time', 1).otherwise(0))
df = df.withColumn('drawn by agreement', F.when(df.Termination == 'drawn by agreement', 1).otherwise(0))
df = df.withColumn('drawn by timeout vs insufficient material', F.when(df.Termination == 'drawn by timeout vs insufficient material', 1).otherwise(0))
df = df.withColumn('drawn by repetition', F.when(df.Termination == 'drawn by repetition', 1).otherwise(0))
df = df.withColumn('won by checkmate', F.when(df.Termination == 'won by checkmate', 1).otherwise(0))
df = df.withColumn('won - game abandoned', F.when(df.Termination == 'won - game abandoned', 1).otherwise(0))
df = df.withColumn('drawn by stalemate', F.when(df.Termination == 'drawn by stalemate', 1).otherwise(0))
df = df.withColumn('drawn by insufficient material', F.when(df.Termination == 'drawn by insufficient material', 1).otherwise(0))
df = df.withColumn('drawn by 50-move rule', F.when(df.Termination == 'drawn by 50-move rule', 1).otherwise(0))
df = df.withColumn('won by resignation', F.when(df.Termination == 'won by resignation', 1).otherwise(0))

In [119]:
df.show()

+-----------+----------+-----+------------------+-------------+-------+--------------+--------------------+---+--------------------+--------+--------+-----------+--------------------+---------+----------+--------+--------------------+------------+--------------------+--------------------+--------------------+--------------------+--------------------+-------------+--------+--------+--------+--------+--------+--------+-------------+-------------+-------------+-------------+-------------+-------------+-----------+-----------+-----------+-----------+-----------+-----------+----------------+----------------+----------------+----------------+----------------+----------------+--------------------+--------------------+--------------------+--------------------+--------+--------+----------------+----------+----------+--------------------+--------------------+--------------------+-----------------------+-----------------------+-----------------------+--------------------+--------------------+----

In [59]:
df = df.drop(*('Link', 'CurrentPosition', 'TimeControl','ECO', 'Result', 'Termination', 'EcoUrl', 'coin_array_18','Moves_clean', 'sf_n1_d22', 'Moves_time', 'parts_idxs', 'sf_n10_d18', 'Moves_sf', 'coin_array',  'Moves', 'Round', 'EndDate','EndTime', 'Tournament_long'))

In [60]:
df.show()

+----------+------------------+-------------+--------------+--------+--------+---------+------------+-------------+-------------+-------------+-------------+-------------+-------------+----------------+----------------+----------------+----------------+----------------+----------------+--------------------+--------+--------+----------------+----------+----------+--------------------+--------------------+--------------------+-----------------------+-----------------------+-----------------------+--------------------+--------------------+--------------------+-----------------------+-----------------------+-----------------------+------------+------------+------------+------------+------------+------------+------------+------------+------------+------------+-------------+-------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+------------------

In [61]:
df = df.drop('Tournament')

In [41]:
columns = ['WhiteElo',
 'BlackElo',
 'Moves_length',
 'coin_w_frac_d',
 'coin_w_frac_m',
 'coin_w_frac_e',
 'coin_b_frac_d',
 'coin_b_frac_m',
 'coin_b_frac_e',
 'coin_w_frac_d_18',
 'coin_w_frac_m_18',
 'coin_w_frac_e_18',
 'coin_b_frac_d_18',
 'coin_b_frac_m_18',
 'coin_b_frac_e_18',
 'Result_w',
 'Result_b',
 'Tournament_is_tt',
 'rate_ratio',
 'rate_prob',
 'mean_prev_scores_d_w',
 'mean_prev_scores_m_w',
 'mean_prev_scores_e_w',
 'mean_prev_scores_d_18_w',
 'mean_prev_scores_m_18_w',
 'mean_prev_scores_e_18_w',
 'mean_prev_scores_d_b',
 'mean_prev_scores_m_b',
 'mean_prev_scores_e_b',
 'mean_prev_scores_d_18_b',
 'mean_prev_scores_m_18_b',
 'mean_prev_scores_e_18_b',
 'mean_times_0',
 'mean_times_1',
 'mean_times_2',
 'mean_times_3',
 'mean_times_4',
 'mean_times_5',
 'mean_times_6',
 'mean_times_7',
 'mean_times_8',
 'mean_times_9',
 'mean_times_10',
 'mean_times_11',
 'mean_prev_times_cd_b',
 'mean_prev_times_ad_b',
 'mean_prev_times_cm_b',
 'mean_prev_times_am_b',
 'mean_prev_times_ce_b',
 'mean_prev_times_ae_b',
 'mean_prev_times_cd_w',
 'mean_prev_times_ad_w',
 'mean_prev_times_cm_w',
 'mean_prev_times_am_w',
 'mean_prev_times_ce_w',
 'mean_prev_times_ae_w',
 'var_time_w',
 'var_time_b',
 'won on time',
 'drawn by agreement',
 'drawn by timeout vs insufficient material',
 'drawn by repetition',
 'won by checkmate',
 'won - game abandoned',
 'drawn by stalemate',
 'drawn by insufficient material',
 'drawn by 50-move rule',
 'won by resignation']

In [134]:
df = df.withColumn('stats_array_', F.array(*columns))

In [135]:
df = df.drop(*columns)

In [58]:
df = df.drop('coin_w_d', 'best_wa', 'coin_w_m', 'coin_w_e', 'coin_b_d', 'coin_b_m', 'coin_b_e', 'coin_w_d_18', 'coin_w_m_18', 'coin_w_e_18', 'coin_b_d_18', 'coin_b_m_18', 'coin_b_e_18', 'stats_array')

In [129]:
df = df.drop(*('Link', 'CurrentPosition', 'EcoUrl', 'Moves', 'Moves_clean', 'sf_n1_d22', 'sf_n10_d18', 'Moves_sf', 'coin_array', 'Data', 'Round', 'StartTime', 'EndDate','EndTime', 'Tournament_long'))

In [130]:
df.show()

+----------+------------------+-------------+-------+--------------+---+--------+--------+-----------+--------------------+------------+-------------+--------+--------+--------+--------+--------+--------+-------------+-------------+-------------+-------------+-------------+-------------+-----------+-----------+-----------+-----------+-----------+-----------+----------------+----------------+----------------+----------------+----------------+----------------+--------------------+--------------------+--------------------+--------+--------+----------------+----------+----------+--------------------+--------------------+--------------------+-----------------------+-----------------------+-----------------------+--------------------+--------------------+--------------------+-----------------------+-----------------------+-----------------------+------------+------------+------------+------------+------------+------------+------------+------------+------------+------------+-------------+----

In [131]:
df = df.withColumn('won_resignation', F.col('won by resignation'))
df = df.withColumn('drawn_50move', F.col('drawn by 50-move rule'))
df = df.withColumn('drawn_insmaterial', F.col('drawn by insufficient material'))
df = df.withColumn('drawn_stalemate', F.col('drawn by stalemate'))
df = df.withColumn('won_abandoned', F.col('won - game abandoned'))
df = df.withColumn('won_checkmate', F.col('won by checkmate'))
df = df.withColumn('drawn_repet', F.col('drawn by repetition'))
df = df.withColumn('drawn_timeout', F.col('drawn by timeout vs insufficient material'))
df = df.withColumn('drawn_agreement', F.col('drawn by agreement'))
df = df.withColumn('won_time', F.col('won on time'))

In [132]:
df.show()

+----------+------------------+-------------+-------+--------------+---+--------+--------+-----------+--------------------+------------+-------------+--------+--------+--------+--------+--------+--------+-------------+-------------+-------------+-------------+-------------+-------------+-----------+-----------+-----------+-----------+-----------+-----------+----------------+----------------+----------------+----------------+----------------+----------------+--------------------+--------------------+--------------------+--------+--------+----------------+----------+----------+--------------------+--------------------+--------------------+-----------------------+-----------------------+-----------------------+--------------------+--------------------+--------------------+-----------------------+-----------------------+-----------------------+------------+------------+------------+------------+------------+------------+------------+------------+------------+------------+-------------+----

In [133]:
df = df.drop('won on time',
 'drawn by agreement',
 'drawn by timeout vs insufficient material',
 'drawn by repetition',
 'won by checkmate',
 'won - game abandoned',
 'drawn by stalemate',
 'drawn by insufficient material',
 'drawn by 50-move rule',
 'won by resignation')

In [134]:
df.show()

+----------+------------------+-------------+-------+--------------+---+--------+--------+-----------+--------------------+------------+-------------+--------+--------+--------+--------+--------+--------+-------------+-------------+-------------+-------------+-------------+-------------+-----------+-----------+-----------+-----------+-----------+-----------+----------------+----------------+----------------+----------------+----------------+----------------+--------------------+--------------------+--------------------+--------+--------+----------------+----------+----------+--------------------+--------------------+--------------------+-----------------------+-----------------------+-----------------------+--------------------+--------------------+--------------------+-----------------------+-----------------------+-----------------------+------------+------------+------------+------------+------------+------------+------------+------------+------------+------------+-------------+----

### Дисперсия по времени

In [120]:
from pyspark.sql.functions import variance
import re

def motion_times_disp(timeControl, moves, parts):

    all_time_w = timeControl
    inc = 0
    
    if len(all_time_w) > 3:
        inc = float(all_time_w[4])
        all_time_w = float(all_time_w[0:3])    
    else:
        all_time_w = float(all_time_w)
        
    all_time_b = all_time_w
    time_pattern = r'\d+:\d+:\d+\.?\d?'
    times = re.findall(time_pattern, moves)
    times_white_m, times_white_e, times_white_d, times_black_d, times_black_m, times_black_e = [], [], [], [], [], []
    
    for i, time in enumerate(times):
        hours, minutes, seconds = time.split(':')
        now_time = float(hours)*3600 + float(minutes)*60 + float(seconds) 
        if not i%2:
            all_time_w += inc
            if float(parts[0]) > i:
                times_white_d.append(all_time_w - now_time)
            elif float(parts[1]) > i:
                times_white_m.append(all_time_w - now_time)
                
            else:
                times_white_e.append(all_time_w - now_time)
            all_time_w = now_time 
        else: 
            all_time_b += inc
            if float(parts[0]) > i:
                times_black_d.append(all_time_b - now_time)
            elif float(parts[1]) > i:
                times_black_m.append(all_time_b - now_time)
            else:
                times_black_e.append(all_time_b - now_time)
            all_time_b = now_time

    times_white_d, times_white_m, times_white_e, times_black_d, times_black_m, times_black_e
    
    mean_times_white_d = sum(times_white_d)/max(len(times_white_d), 1)
    mean_times_white_m = sum(times_white_m)/max(len(times_white_m), 1)
    mean_times_white_e = sum(times_white_e)/max(len(times_white_e), 1)
    mean_times_black_d = sum(times_black_d)/max(len(times_black_d), 1)
    mean_times_black_m = sum(times_black_m)/max(len(times_black_m), 1)
    mean_times_black_e = sum(times_black_e)/max(len(times_black_e), 1)

    sum_diff_w_d = 0
    sum_diff_w_m = 0
    sum_diff_w_e = 0
    sum_diff_b_d = 0
    sum_diff_b_m = 0
    sum_diff_b_e = 0
    
    for i in range(len(times_white_d)):
        sum_diff_w_d += (times_white_d[i] - mean_times_white_d)**2
    for i in range(len(times_white_m)):
        sum_diff_w_m += (times_white_m[i] - mean_times_white_m)**2
    for i in range(len(times_white_e)):
        sum_diff_w_e += (times_white_e[i] - mean_times_white_e)**2
    for i in range(len(times_black_d)):
        sum_diff_b_d += (times_black_d[i] - mean_times_black_d)**2
    for i in range(len(times_black_m)):
        sum_diff_b_m += (times_black_m[i] - mean_times_black_m)**2
    for i in range(len(times_black_e)):
        sum_diff_b_e += (times_black_e[i] - mean_times_black_e)**2
    
    return [sum_diff_w_d/max(len(times_white_d), 1), sum_diff_w_m/max(len(times_white_m), 1), \
            sum_diff_w_e/max(len(times_white_e), 1), sum_diff_b_d/max(len(times_black_d), 1), \
            sum_diff_b_m/max(len(times_black_m), 1), sum_diff_b_e/max(len(times_black_e), 1) ]

In [121]:
disp_times_udf = udf(motion_times_disp, ArrayType(FloatType()))
df = df.withColumn('var_time_w_d', disp_times_udf(df['TimeControl'], df['Moves'], df['parts_idxs'])[0])
df = df.withColumn('var_time_w_m', disp_times_udf(df['TimeControl'], df['Moves'], df['parts_idxs'])[1])
df = df.withColumn('var_time_w_e', disp_times_udf(df['TimeControl'], df['Moves'], df['parts_idxs'])[2])
df = df.withColumn('var_time_b_d', disp_times_udf(df['TimeControl'], df['Moves'], df['parts_idxs'])[3])
df = df.withColumn('var_time_b_m', disp_times_udf(df['TimeControl'], df['Moves'], df['parts_idxs'])[4])
df = df.withColumn('var_time_b_e', disp_times_udf(df['TimeControl'], df['Moves'], df['parts_idxs'])[5])

In [122]:
df.show()

+-----------+----------+-----+------------------+-------------+-------+--------------+--------------------+---+--------------------+--------+--------+-----------+--------------------+---------+----------+--------+--------------------+------------+--------------------+--------------------+--------------------+--------------------+--------------------+-------------+--------+--------+--------+--------+--------+--------+-------------+-------------+-------------+-------------+-------------+-------------+-----------+-----------+-----------+-----------+-----------+-----------+----------------+----------------+----------------+----------------+----------------+----------------+--------------------+--------------------+--------------------+--------------------+--------+--------+----------------+----------+----------+--------------------+--------------------+--------------------+-----------------------+-----------------------+-----------------------+--------------------+--------------------+----

### Ошибки и зевки

In [123]:
def blunders(sf):
    real = sf
    print(real)
    
    blunders_w = 0
    best_move_w = 0
    blunders_b = 0
    best_move_b = 0
    
    if len(real) < 1: 
        return [best_move_w, best_move_b, blunders_w, blunders_b]

    for i in range(1, len(real)):
        if real[i][0]['_2'] is None: 
            return [best_move_w, best_move_b, blunders_w, blunders_b]
        else: 
            cp = real[i-1][0]['_2'] - real[i][0]['_2']
        if not i%2:
            if cp > 200:
                best_move_w += 1
            if cp < -200: 
                blunders_w += 1
        else:
            if cp > 200:
                best_move_b += 1
            if cp < -200: 
                blunders_b += 1
                
    return  [best_move_w, best_move_b, blunders_w, blunders_b]

In [124]:
blunders_udf = udf(blunders, ArrayType(IntegerType()))
df = df.withColumn('best_w', blunders_udf(df['sf_n1_d22'])[0])
df = df.withColumn('blunders_w',blunders_udf(df['sf_n1_d22'])[2])
df = df.withColumn('best_b', blunders_udf(df['sf_n1_d22'])[1])
df = df.withColumn('blunders_b', blunders_udf(df['sf_n1_d22'])[3])

In [125]:
df.show(truncate=False)

+-----------+----------+-----+------------------+-------------+-------+--------------+-----------------------------------------------------+---+-----------------------------------------------------------------------------+--------+--------+-----------+------------------------------+---------+----------+--------+---------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------

### Средняя потеря в сантипешках

In [126]:
def mean_cp(sf):
    real = sf
    
    cp_w = []
    cp_b = []
    
    if len(real) < 1: 
        return [0, 0]

    for i in range(1, len(real)):
        if real[i][0]['_2'] is None: 
            return [sum(cp_w)/max(1, len(cp_w)), sum(cp_b)/max(1, len(cp_b))]
        else: 
            cp = real[i-1][0]['_2'] - real[i][0]['_2']
        if not i%2:
            cp_w.append(cp)
        else:
            cp_b.append(cp)
                
    return  [sum(cp_w)/max(1, len(cp_w)), sum(cp_b)/max(1, len(cp_b))]

In [127]:
mean_cp_udf = udf(mean_cp, ArrayType(FloatType()))
df = df.withColumn('mean_cp_w', mean_cp_udf(df['sf_n1_d22'])[0])
df = df.withColumn('mean_cp_b', mean_cp_udf(df['sf_n1_d22'])[1])

In [128]:
df.show()

+-----------+----------+-----+------------------+-------------+-------+--------------+--------------------+---+--------------------+--------+--------+-----------+--------------------+---------+----------+--------+--------------------+------------+--------------------+--------------------+--------------------+--------------------+--------------------+-------------+--------+--------+--------+--------+--------+--------+-------------+-------------+-------------+-------------+-------------+-------------+-----------+-----------+-----------+-----------+-----------+-----------+----------------+----------------+----------------+----------------+----------------+----------------+--------------------+--------------------+--------------------+--------------------+--------+--------+----------------+----------+----------+--------------------+--------------------+--------------------+-----------------------+-----------------------+-----------------------+--------------------+--------------------+----

### Датасет для анализа по турнирам

In [9]:
df_filtered = df.filter((F.col('Tournament_long') != "") & (F.col('Tournament_long').isNotNull()))

In [ ]:
df_filtered.show()

+-----------+----------+-----+---------------+-------------------+-------+--------------+--------------------+---+--------------------+--------+--------+-----------+--------------------+---------+----------+--------+--------------------+------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------+--------+--------+--------+--------+--------+--------+-------------+-------------+-------------+-------------+-------------+-------------+--------+--------+
|       Link|      Date|Round|          White|              Black| Result|    Tournament|     CurrentPosition|ECO|              ECOUrl|WhiteElo|BlackElo|TimeControl|         Termination|StartTime|   EndDate| EndTime|               Moves|Moves_length|     Tournament_long|         Moves_clean|           sf_n1_d22|          sf_n10_d18|            Moves_sf|    parts_idxs|coin_w_d|coin_w_m|coin_w_e|coin_b_d|coin_b_m|coin_b_e|coin_w_frac_d|coin_w_frac_m|coin_w_frac_e|coin_b_

In [ ]:
df_filtered_2 = df.select("Date",  "White", "Black", "Tournament", "WhiteElo", "BlackELo", "StartTime", "Tournament_long","coin_w_frac_d", "coin_w_frac_m", "coin_w_frac_e", "coin_b_frac_d", "coin_b_frac_m", "coin_b_frac_e", "Result_w", "Result_b")

In [30]:
df_filtered_2.show()

+----------+---------------+-------------------+--------------+--------+--------+---------+--------------------+-------------+-------------+-------------+-------------+-------------+-------------+--------+--------+
|      Date|          White|              Black|    Tournament|WhiteElo|BlackELo|StartTime|     Tournament_long|coin_w_frac_d|coin_w_frac_m|coin_w_frac_e|coin_b_frac_d|coin_b_frac_m|coin_b_frac_e|Result_w|Result_b|
+----------+---------------+-------------------+--------------+--------+--------+---------+--------------------+-------------+-------------+-------------+-------------+-------------+-------------+--------+--------+
|2021.03.23|         zala02|       ChessAurochs|titled-tuesday|    2194|    2554| 17:00:06|-titled-tuesday-b...|          0.8|          0.0|   0.44444445|          0.4|   0.44444445|   0.44444445|     0.0|     1.0|
|2021.03.23|      GMSrinath|           Cryptinz|titled-tuesday|    2808|    2534| 17:10:19|-titled-tuesday-b...|          0.4|    0.4347826|